In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/online-retail/Online_Retail.csv


In [2]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
# Assuming 'online_retail.csv' is your dataset file
df = pd.read_csv('/kaggle/input/online-retail/Online_Retail.csv', encoding='unicode_escape')

In [4]:
# Display the columns of the data
print("Columns:")
print(df.columns)

# Display a portion of the data
print("\nSample of the Data:")
print(df.head())

# Check for unique values in each column
print("\nUnique Values in Each Column:")
for column in df.columns:
    unique_values = df[column].nunique()
    print(f"{column}: {unique_values} unique values")

# Check for missing values
print("\nMissing Values:")
print(df.isnull().sum())

Columns:
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

Sample of the Data:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

    InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/10 8:26       2.55     17850.0  United Kingdom  
1  12/1/10 8:26       3.39     17850.0  United Kingdom  
2  12/1/10 8:26       2.75     17850.0  United Kingdom  
3  12/1/10 8:26       3.39     17850.0  United Kingdom  
4  12/1/10 8:26       3.39     17850.0  United Kingdom  

Unique Values in Each Column:
InvoiceNo: 25900 uniq

In [5]:
# Strip strings
df['Description'] = df['Description'].str.strip()
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# Drop rows having no InvoiceID
df = df.dropna(subset=['InvoiceNo'])

# Drop invoices done with a credit (denoted by 'C' in InvoiceNo)
df = df[~df['InvoiceNo'].str.contains('C')]

# Display a portion of the cleaned data
print("\nCleaned Data:")
print(df.head())


Cleaned Data:
  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

    InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/10 8:26       2.55     17850.0  United Kingdom  
1  12/1/10 8:26       3.39     17850.0  United Kingdom  
2  12/1/10 8:26       2.75     17850.0  United Kingdom  
3  12/1/10 8:26       3.39     17850.0  United Kingdom  
4  12/1/10 8:26       3.39     17850.0  United Kingdom  


In [6]:
# Replace 'Region' with the actual column name if it's different
region_column = 'Country'

# Filter the data according to the region
region_data = df[df[region_column] == 'United Kingdom']

# Group the data according to 'InvoiceNo' and 'Description' and sum according to 'Quantity'
grouped_data = region_data.groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index()

# Fill NaN values with 0
grouped_data = grouped_data.fillna(0)

# Set 'InvoiceNo' as the new index
grouped_data.set_index('InvoiceNo', inplace=True)

# Display the resulting DataFrame
print("\nTransformed Data:")
print(grouped_data.head())


Transformed Data:
Description  *Boombox Ipod Classic  *USB Office Mirror Ball  \
InvoiceNo                                                     
536365                         0.0                      0.0   
536366                         0.0                      0.0   
536367                         0.0                      0.0   
536368                         0.0                      0.0   
536369                         0.0                      0.0   

Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536365                          0.0                         0.0   
536366                          0.0                         0.0   
536367                          0.0                         0.0   
536368                          0.0                         0.0   
536369                          0.0                         0.0   

Description  12 DAISY PEGS IN WOOD BOX  12 EGG HOUSE PAINTED WOOD  \


In [7]:
# Assuming 'data' is your DataFrame
# Line 5: Filter the data for the country 'France'
test = df[df['Country'] == "France"]

# Line 6: Group the data by 'InvoiceNo' and 'Description'
# Line 7: Select the 'Quantity' column from the grouped data
test = test.groupby(['InvoiceNo', 'Description'])['Quantity']

# Line 8: Sum the quantities for each group
test = test.sum()

# Line 9: Unstack the grouped data, pivot 'Description' to columns
test = test.unstack()

test = test.loc[:, (test != 0).any(axis=0)]

# Line 10: Reset the index to make 'InvoiceNo' a regular column
test = test.reset_index()

# Line 11: Fill NaN values with 0
test = test.fillna(0)

# Line 12: Set 'InvoiceNo' as the new index
test = test.set_index('InvoiceNo')

# Line 13: Display the resulting DataFrame
test.head()


Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536852,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537065,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537463,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:

def one_hot_encode(df):
    """
    One-hot encode a DataFrame by replacing values greater than 0 with 1 and values less than or equal to 0 with 0.

    Parameters:
    - df: DataFrame, the input DataFrame

    Returns:
    - encoded_df: DataFrame, the one-hot encoded DataFrame
    """
    encoded_df = df.applymap(lambda x: 1 if x > 0 else 0)
    return encoded_df

# Example usage:
# Assuming 'test' is your DataFrame
encoded_test = one_hot_encode(test)

# Display the one-hot encoded DataFrame
print("\nOne-Hot Encoded Data:")
print(encoded_test.head())



One-Hot Encoded Data:
Description  10 COLOUR SPACEBOY PEN  12 COLOURED PARTY BALLOONS  \
InvoiceNo                                                         
536370                            0                           0   
536852                            0                           0   
536974                            0                           0   
537065                            0                           0   
537463                            0                           0   

Description  12 EGG HOUSE PAINTED WOOD  12 MESSAGE CARDS WITH ENVELOPES  \
InvoiceNo                                                                 
536370                               0                                0   
536852                               0                                0   
536974                               0                                0   
537065                               0                                0   
537463                               0                   

In [9]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Assuming 'encoded_test' is your one-hot encoded DataFrame
# You can use the one-hot encoded DataFrame obtained from the previous steps

# Set the minimum support threshold (adjust as needed)
min_support = 0.05

# Get frequent itemsets using Apriori
frequent_itemsets = apriori(encoded_test, min_support=min_support, use_colnames=True)

# Display frequent itemsets
print("\nFrequent Itemsets:")
print(frequent_itemsets)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Display generated rules
print("\nAssociation Rules:")
print(rules)



Frequent Itemsets:
      support                                           itemsets
0    0.071429                      (4 TRADITIONAL SPINNING TOPS)
1    0.096939                       (ALARM CLOCK BAKELIKE GREEN)
2    0.102041                        (ALARM CLOCK BAKELIKE PINK)
3    0.094388                         (ALARM CLOCK BAKELIKE RED)
4    0.068878                       (ASSORTED COLOUR MINI CASES)
..        ...                                                ...
190  0.102041  (POSTAGE, SET/6 RED SPOTTY PAPER PLATES, SET/6...
191  0.099490  (SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...
192  0.056122  (POSTAGE, ALARM CLOCK BAKELIKE PINK, ALARM CLO...
193  0.053571  (POSTAGE, PLASTERS IN TIN CIRCUS PARADE, PLAST...
194  0.081633  (POSTAGE, SET/6 RED SPOTTY PAPER PLATES, SET/6...

[195 rows x 2 columns]

Association Rules:
                                           antecedents  \
0                        (4 TRADITIONAL SPINNING TOPS)   
1                                       

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [10]:
# Assuming 'frequent_itemsets' is the DataFrame obtained from the apriori step
# You need to have the frequent itemsets DataFrame obtained from the previous apriori step

# Set the minimum confidence level (adjust as needed)
min_confidence = 0.7

# Generate association rules with minimum confidence
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=min_confidence)

# Sort and display the rules
sorted_rules = rules.sort_values(by='confidence', ascending=False)

# Display the sorted rules
print("\nSorted Association Rules:")
print(sorted_rules)



Sorted Association Rules:
                                           antecedents  \
27                        (JUMBO BAG WOODLAND ANIMALS)   
101  (PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...   
104  (RED TOADSTOOL LED NIGHT LIGHT, PLASTERS IN TI...   
119  (SET/6 RED SPOTTY PAPER CUPS, SET/20 RED RETRO...   
118  (SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...   
..                                                 ...   
42                         (PLASTERS IN TIN STRONGMAN)   
21                              (DOLLY GIRL LUNCH BOX)   
106             (POSTAGE, SET/6 RED SPOTTY PAPER CUPS)   
129  (ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...   
89                          (ALARM CLOCK BAKELIKE RED)   

                              consequents  antecedent support  \
27                              (POSTAGE)            0.076531   
101                             (POSTAGE)            0.051020   
104                             (POSTAGE)            0.053571   
119       (SET/6